In [11]:
import pandas as pd
import json
import requests
from typing import Dict, Any

SCHEDULE_URL_FORMAT = 'https://www.jfa.jp/national_team/u24_2021/tokyo_olympic_2020/group{}/match/schedule.json'
# 'http://www.jfa.jp/match/acl_2021/group{}/match/schedule.json'
SCHEDULE_CONTAINER_NAME = 'matchScheduleList'
SCHEDULE_LIST_NAME = 'matchSchedule'
CSV_FILENAME = '../csv/2021_allmatch_result-Olympic_GL.csv'
JSON_FILENAME = '../docs/json/olympic_points.json'


REPLACE_KEY_DICT = {
    'match_date': 'matchDateJpn',
    'section_no': 'matchTypeName',
    'start_time': 'matchTimeJpn',
    'stadium': 'venue',
    'home_team': 'homeTeamName',
    'away_team': 'awayTeamName',
    'status': 'matchStatus',
    'matchNumber': 'matchNumber'
}
SCORE_DATA_KEY_LIST = {
    'home_goal': 'homeScore',
    'away_goal': 'awayScore',
    'extraTime': 'exMatch'
}

def read_match_json(group: str) -> Dict[str, Any]:
    """各グループの試合リスト情報をjfaのJSON形式で返す
    """
    _url = SCHEDULE_URL_FORMAT.format(group)
    print(f'access {_url}...')
    return json.loads(requests.get(_url).text)

def read_match_df(group: str) -> pd.DataFrame:
    """各グループの試合リスト情報を自分たちのDataFrame形式で返す
    JFA形式のJSONは、1試合の情報が下記のような内容
    {'matchTypeName': '第1節',
     'matchNumber': '1',  # どうやら、Competitionで通しの番号
     'matchDate': '2021/07/22',  # 未使用
     'matchDateJpn': '2021/07/22',
     'matchDateWeek': '木',  # 未使用
     'matchTime': '20:00',  # 未使用
     'matchTimeJpn': '20:00',
     'venue': '東京スタジアム',
     'venueFullName': '東京／東京スタジアム',  # 未使用
     'homeTeamName': '日本',
     'homeTeamQualificationDescription': '',  # 未使用
     'awayTeamName': '南アフリカ',
     'awayTeamQualificationDescription': '',  # 未使用
     'score': {
         'homeWinFlag': False,  # 未使用
         'awayWinFlag': False,  # 未使用
         'homeScore': '',
         'awayScore': '',
         'homeTeamScore1st': '',  # 未使用
         'awayTeamScore1st': '',  # 未使用
         'homeTeamScore2nd': '',  # 未使用
         'awayTeamScore2nd': '',  # 未使用
         'exMatch': False
     },
     'scorer': {
         'homeScorer': [],  # 未使用
         'awayScorer': []  # 未使用
     },
     'matchStatus': '',
     'officialReportURL': ''  # 未使用
    }
    """
    match_list = read_match_json(group)[SCHEDULE_CONTAINER_NAME][SCHEDULE_LIST_NAME]
    # print(match_list)
    result_list = []
    match_index_dict = {}
    for _match_data in match_list:
        _row = {}
        _row['group'] = group
        for (target_key, org_key) in REPLACE_KEY_DICT.items():
            _row[target_key] = _match_data[org_key]
        for (target_key, org_key) in SCORE_DATA_KEY_LIST.items():
            _row[target_key] = _match_data['score'][org_key]
        section_no = _row['section_no']
        if section_no not in match_index_dict:
            match_index_dict[section_no] = 1
        else:
            match_index_dict[section_no] += 1
        _row['match_index_in_section'] = match_index_dict[section_no]
        result_list.append(_row)

    return pd.DataFrame(result_list)

match_df = pd.DataFrame()
for group in ['A', 'B', 'C', 'D']:
    match_df = pd.concat([match_df, read_match_df(group)])

match_df = match_df.sort_values(['group', 'section_no', 'match_index_in_section']).reset_index(drop=True)
match_df.to_csv(CSV_FILENAME)
match_df

access https://www.jfa.jp/national_team/u24_2021/tokyo_olympic_2020/groupA/match/schedule.json...
access https://www.jfa.jp/national_team/u24_2021/tokyo_olympic_2020/groupB/match/schedule.json...
access https://www.jfa.jp/national_team/u24_2021/tokyo_olympic_2020/groupC/match/schedule.json...
access https://www.jfa.jp/national_team/u24_2021/tokyo_olympic_2020/groupD/match/schedule.json...


,group,match_date,section_no,start_time,stadium,home_team,away_team,status,matchNumber,home_goal,away_goal,extraTime,match_index_in_section
0,A,2021/07/22,第1節,20:00,東京スタジアム,日本,南アフリカ,,1,,,False,1
1,A,2021/07/22,第1節,17:00,東京スタジアム,メキシコ,フランス,,2,,,False,2
2,A,2021/07/25,第2節,20:00,埼玉スタジアム２００２,日本,メキシコ,,9,,,False,1
3,A,2021/07/25,第2節,17:00,埼玉スタジアム２００２,フランス,南アフリカ,,10,,,False,2
4,A,2021/07/28,第3節,20:30,横浜国際総合競技場,フランス,日本,,17,,,False,1
5,A,2021/07/28,第3節,20:30,札幌ドーム,南アフリカ,メキシコ,,18,,,False,2
6,B,2021/07/22,第1節,17:00,茨城カシマスタジアム,ニュージーランド,韓国,,3,,,False,1
7,B,2021/07/22,第1節,20:00,茨城カシマスタジアム,ホンジュラス,ルーマニア,,4,,,False,2
8,B,2021/07/25,第2節,17:00,茨城カシマスタジアム,ニュージーランド,ホンジュラス,,11,,,False,1
9,B,2021/07/25,第2節,20:00,茨城カシマスタジアム,ルーマニア,韓国,,12,,,False,2
